In [95]:
import data_loader
import numpy as np

In [96]:
# df = data_loader.load_train_data("data/adult.data")
df = np.genfromtxt("data/adult.data",delimiter=",", dtype="|U", autostrip=True)
df.shape

(32561, 15)

In [97]:
trdf, valdf = data_loader.load_train_data("data/adult.data", is_df=True)

In [105]:


catcols = [1,3,5,6,7,8,9,13,14]
trdf[catcols] = trdf[catcols].astype('category')
trdf[catcols] = trdf[catcols].apply(lambda x: x.cat.codes)
# trdf
# trdf[1].cat.codes
trdf[14].shape

(29315,)

In [110]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
learner = model.fit(trdf.iloc[:,0:14], trdf.iloc[:,14])


In [112]:

catcols = [1,3,5,6,7,8,9,13,14]
valdf[catcols] = valdf[catcols].astype('category')
valdf[catcols] = valdf[catcols].apply(lambda x: x.cat.codes)
# trdf
# trdf[1].cat.codes
trdf[14].shape

(29315,)

In [113]:
learner.predict(valdf.iloc[:,0:14])

array([0, 1, 1, ..., 0, 0, 1], dtype=int8)

In [114]:
learner.score(valdf.iloc[:,0:14], valdf.iloc[:,14])

0.8009858287122612